In [8]:
from bs4 import BeautifulSoup
import os
import random
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
import time


In [9]:
url = 'https://supermarket-scrape-website.vercel.app/scroll/'

In [10]:
options = Options()
options.headless = True
options.add_argument("--window-size=1920,1200")
driver = webdriver.Chrome(options=options)
driver.get(url)
wait = WebDriverWait(driver, 10)  # Waits for up to 10 seconds

In [11]:
categories = driver.find_elements(By.CLASS_NAME, 'category')

In [12]:
folder_path = './csv_dumps/'
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

In [13]:
def extract_products (soup):
    products_list = pd.DataFrame()
    products = soup.find_all(class_='product')
    for product in products:
        name = product.find(class_='product_name').text
        price = product.find(class_='product_price').text
        products_list = pd.concat([products_list, pd.DataFrame([{'name' : name.replace('\n', ' ').replace('\r', ''), 'price' : price}])])
    return products_list

In [14]:
for category in categories:
    try :
        category.click()
        all_products = pd.DataFrame()

        print (f'Extracting Category {category.text}')
        time.sleep(10)

        last_height = driver.execute_script("return document.body.scrollHeight")
        while True:
            for i in range(1, 11):
                driver.execute_script(f"window.scrollTo(0, document.body.scrollHeight*{i/10});")
                time.sleep(3) 
            # driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)

            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break  
            last_height = new_height
        
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')
        all_products = pd.concat([all_products, extract_products(soup)])
        time.sleep(random.uniform(1, 5))
        all_products.to_csv(f'csv_dumps/{category.text}.csv', index=False)
    except Exception as e:
        print(e)
    

Extracting Category Confectionery
Extracting Category Delicatessen
Extracting Category Household
Extracting Category Wine Cellar
Extracting Category Fruits & Vegetables
Extracting Category Organic
Extracting Category Pets
Extracting Category Chilled & Dairy
Extracting Category Fish
Extracting Category Butcher
Message: element not interactable: Element is not currently visible and may not be manipulated
  (Session info: chrome=119.0.6045.160)
Stacktrace:
	GetHandleVerifier [0x00007FF6AFE882B2+55298]
	(No symbol) [0x00007FF6AFDF5E02]
	(No symbol) [0x00007FF6AFCB05AB]
	(No symbol) [0x00007FF6AFCB509C]
	(No symbol) [0x00007FF6AFCB6FDF]
	(No symbol) [0x00007FF6AFCB70D0]
	(No symbol) [0x00007FF6AFCF515C]
	(No symbol) [0x00007FF6AFCE8BD1]
	(No symbol) [0x00007FF6AFD120AA]
	(No symbol) [0x00007FF6AFCE80CF]
	(No symbol) [0x00007FF6AFD122C0]
	(No symbol) [0x00007FF6AFD2AAA4]
	(No symbol) [0x00007FF6AFD11E83]
	(No symbol) [0x00007FF6AFCE670A]
	(No symbol) [0x00007FF6AFCE7964]
	GetHandleVerifier [

In [15]:
driver.quit()

In [16]:
if not os.path.exists('csv_final'):
    os.makedirs('csv_final')

final_file = pd.DataFrame()

source_folder = './csv_dumps/'
files = os.listdir(f'{source_folder}')


In [17]:
for file in files:
    data = pd.read_csv(f'{source_folder}/{file}')
    data['category'] = file.replace('.csv', '')
    final_file = pd.concat([final_file, data])

In [18]:
final_file.to_csv('csv_final/final_products.csv', index=False)